In [ ]:
import psycopg2
from psycopg2 import sql
from psycopg2 import Error

In [ ]:
DB_HOST = 'pavlov.ing.puc.cl'
DB_PORT = '5432'
DB_USER = 'grupo64'
DB_PASSWORD = 'grupo64'
DB_NAME = 'grupo64e2'

In [ ]:
# Ruta del archivo CSV
archivo_csv = '../datos e2/restaurantes2.csv'

# Configuración de la conexión
try:
    conexion = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    # Crear un cursor
    cursor = conexion.cursor()

    # Dropear la tabla si ya existe
    drop_table_query = """
    DROP TABLE IF EXISTS restaurantes;
    DROP TABLE IF EXISTS sucursales;
    """
    cursor.execute(drop_table_query)
    print("Tabla restaurantes, y sucurales eliminadas si existian")

    # Crear la tabla (ajusta esto según tu esquema de datos)
    crear_tabla_query = """
    CREATE TABLE IF NOT EXISTS restaurantes (
        id SERIAL,
        nombre varchar   NOT NULL,
        vigente bool   NOT NULL,
        estilo varchar   NOT NULL,
        CONSTRAINT pk_rest PRIMARY KEY (
            id))
    """
    cursor.execute(crear_tabla_query)
    conexion.commit()

    crear_tabla_query_sucursal = """
    CREATE TABLE IF NOT EXISTS sucursales (
        id SERIAL,
        sucursal varchar NOT NULL,
        direccion varchar   NOT NULL,
        telefono varchar    NOT NULL,
        area varchar NOT NULL,
        CONSTRAINT pk_sucr PRIMARY KEY (
            id))
    """
    cursor.execute(crear_tabla_query_sucursal)
    conexion.commit()


    # Preparar el query de inserción
    insercion_query = """
    INSERT INTO restaurantes (nombre, vigente, estilo)
    VALUES (%s, %s, %s);
    """
    # Preparar el query de inserción sucursal
    insercion_query_sucursal ="""
    INSERT INTO sucursales (sucursal, direccion, telefono, area)
    VALUES (%s, %s, %s, %s);
    """


    # Leer y procesar el archivo CSV
    with open(archivo_csv, 'r') as file:
        lista_restaurant = []
        restaurant_actual = ""
        next(file)

        for linea in file:
            restaurant_actual += linea.strip()
            valores = restaurant_actual.split(';')
            if len(valores) == 8:
                lista_restaurant.append(valores)
                restaurant_actual = ""
            elif len(valores) > 8:
                print(f"Línea malformada: {restaurant_actual}")
                restaurant_actual = ""

        for restaurant in lista_restaurant:
            try:
                cursor.execute(insercion_query, (restaurant[0], restaurant[1], restaurant[2]))
                cursor.execute(insercion_query_sucursal, (restaurant[4], restaurant[5], restaurant[6], restaurant[7])) 
            except Error as e:
                    print(f"Error al insertar la línea {linea}: {e}")
                    conexion.rollback()
                    continue

    # Confirmar la inserción
    conexion.commit()

except Error as e:
    print(f"Error de conexión o durante la operación en la base de datos: {e}")

finally:

    if cursor:
        cursor.close()
    if conexion:
        conexion.close()

In [64]:
try:
    conexion = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )

    # Crear un cursor
    cursor = conexion.cursor()
    # Dropear la tabla si ya existe
    drop_table_query = """
    DROP TABLE IF EXISTS asociado_a;
    """
    cursor.execute(drop_table_query)
    print("Tabla asociado_a eliminadas si existian")
    # Consulta para seleccionar todos los datos de la tabla 'clientes'
    seleccionar_query = "SELECT * FROM sucursales"
    
    cursor.execute(seleccionar_query)
    registros = cursor.fetchall()

    crear_tabla_query_relation = """
    CREATE TABLE IF NOT EXISTS asociado_a (
        id_restaurant INT NOT NULL,
        id_sucursal INT NOT NULL)
    """
    cursor.execute(crear_tabla_query_relation)
    conexion.commit()    


    # Preparar el query de inserción relación
    insercion_query_relation = """
    INSERT INTO asociado_a (id_restaurant, id_sucursal)
    VALUES (%s, %s);
    """

    # Imprimir los registros
    with open(archivo_csv, 'r') as file:
        lista_restaurant = []
        restaurant_actual = ""
        lista_completada = []
        next(file)
        n= 0
        for linea in file:
            restaurant_actual += linea.strip()
            valores = restaurant_actual.split(';')
            if len(valores) == 8:
                lista_restaurant.append(valores)
                restaurant_actual = ""
            elif len(valores) > 8:
                print(f"Línea malformada: {restaurant_actual}")
                restaurant_actual = ""
    for registro in registros:
        for restaurant in lista_restaurant:
            
            if registro[2] == restaurant[5] and registro[0] not in lista_completada:
                cursor.execute(insercion_query_relation, (lista_restaurant.index(restaurant), registro[0]))
                lista_completada.append(registro[0])
    conexion.commit()

    
except Error as e:
    print(f"Error de conexión o durante la operación en la base de datos: {e}")

finally:

    if cursor:
        cursor.close()
    if conexion:
        conexion.close()

Tabla asociado_a eliminadas si existian


In [ ]:
try:
    conexion = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    # Crear un cursor
    cursor = conexion.cursor()
    # Consulta para seleccionar todos los datos de la tabla 'clientes'
    seleccionar_query = "SELECT * FROM restaurantes;"
    
    cursor.execute(seleccionar_query)
    # Obtener todos los registros
    registros = cursor.fetchall()

    # Imprimir los registros
    for registro in registros:
        print(registro)

except Error as e:
    print(f"Error de conexión o durante la operación en la base de datos: {e}")

finally:

    if cursor:
        cursor.close()
    if conexion:
        conexion.close()

In [ ]:
try:
    conexion = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    # Crear un cursor
    cursor = conexion.cursor()
    # Consulta para seleccionar todos los datos de la tabla 'clientes'
    seleccionar_query = "SELECT * FROM sucursales;"
    
    cursor.execute(seleccionar_query)
    # Obtener todos los registros
    registros = cursor.fetchall()

    # Imprimir los registros
    for registro in registros:
        print(registro)

except Error as e:
    print(f"Error de conexión o durante la operación en la base de datos: {e}")

finally:

    if cursor:
        cursor.close()
    if conexion:
        conexion.close()

In [65]:
try:
    conexion = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    # Crear un cursor
    cursor = conexion.cursor()
    # Consulta para seleccionar todos los datos de la tabla 'clientes'
    seleccionar_query = "SELECT * FROM asociado_a;"
    
    cursor.execute(seleccionar_query)
    # Obtener todos los registros
    registros = cursor.fetchall()

    # Imprimir los registros
    for registro in registros:
        print(registro)

except Error as e:
    print(f"Error de conexión o durante la operación en la base de datos: {e}")

finally:

    if cursor:
        cursor.close()
    if conexion:
        conexion.close()

(0, 1)
(1, 2)
(2, 3)
(3, 4)
(4, 5)
(5, 6)
(6, 7)
(7, 8)
(8, 9)
(9, 10)
(10, 11)
(11, 12)
(12, 13)
(13, 14)
(14, 15)
(15, 16)
(16, 17)
(3, 18)
(18, 19)
(19, 20)
(20, 21)
(21, 22)
(11, 23)
(23, 24)
(24, 25)
(25, 26)
(3, 27)
(27, 28)
(28, 29)
(29, 30)
(30, 31)
(31, 32)
(32, 33)
(10, 34)
(34, 35)
(35, 36)
(36, 37)
(37, 38)
(38, 39)
(18, 40)
(40, 41)
(41, 42)
(24, 43)
(25, 44)
(3, 45)
(27, 46)
(28, 47)
(29, 48)
(30, 49)
(31, 50)
(32, 51)
(10, 52)
(52, 53)
(53, 54)
(24, 55)
(25, 56)
(56, 57)
(57, 58)
(19, 59)
(59, 60)
(36, 61)
(2, 62)
(53, 63)
(24, 64)
(25, 65)
(56, 66)
(57, 67)
(19, 68)
(59, 69)
(36, 70)
(70, 71)
(71, 72)
(72, 73)
(73, 74)
(74, 75)
(75, 76)
(2, 77)
(3, 78)
(78, 79)
(19, 80)
(59, 81)
(81, 82)
(82, 83)
(38, 84)
(18, 85)
(52, 86)
(20, 87)
(21, 88)
(11, 89)
(23, 90)
(13, 91)
(14, 92)
(35, 93)
(36, 94)
(37, 95)
(38, 96)
(18, 97)
(40, 98)
(41, 99)
(24, 100)
(25, 101)
(3, 102)
(27, 103)
(28, 104)
(29, 105)
(30, 106)
(31, 107)
(32, 108)
(10, 109)
(52, 110)
(53, 111)
(24, 112)
(25, 